In [1]:
library(ttutils)
library(modes)
library(caret)

#stringsAsFactors = FALSE ，避免字串存成numeric
data=read.csv('C:\\Users\\coco40725\\Documents\\GitHub\\2nd-ML100Days\\data\\titanic_train.csv',stringsAsFactors = FALSE)

col.type=c()
for (i in 1:ncol(data)) {
  if(is.character(data[,i])==T){
    col.type[i]='character'
  }else if (mean(isInteger(data[,i]),na.rm = T)==1){
    col.type[i]='integer'
  }else{
    col.type[i]='float'
  }
}

data.use=data[,which(col.type!='character')]
data.use=data.use[,-c(1,2)]
data.repl.mean=data.use
data.repl.mode=data.use
#觀察替換不同補缺方式, 對於特徵的影響 (In[4]~In[6], Out[4]~Out[6])
# mean
for (i in 1:ncol(data.repl.mean)) {
  loc.na=is.na(data.repl.mean[,i])
  data.repl.mean[loc.na,i]=mean(data.repl.mean[,i],na.rm = T)
}

# mode
for (j in 1:ncol(data.repl.mode)) {
  loc.na=is.na(data.repl.mode[,j])
  data.repl.mode[loc.na,j]=as.numeric(modes(data.repl.mode[,j])[1,])
}


#使用不同的標準化方式 ( 原值 / 最小最大化 / 標準化 )，搭配羅吉斯迴歸模型，何者效果最好?

###origin
data.repl.mean.d=data.frame(data.repl.mean,survive=as.factor(data$Survived))
data.repl.mode.d=data.frame(data.repl.mode,survive=as.factor(data$Survived))

#origin + mean
train_control = trainControl(method = "cv", number = 10)
fit1.cv=train(survive~ .,
              data = data.repl.mean.d,
              trControl = train_control,
              method = "glm",
              family=binomial(link='logit'))

acc1=fit1.cv$results[,2]

#origin + mode

train_control = trainControl(method = "cv", number = 10)
fit2.cv=train(survive~ .,
              data = data.repl.mode.d,
              trControl = train_control,
              method = "glm",
              family=binomial(link='logit'))

acc2=fit2.cv$results[,2]

## min-max

#min-max + mean
data.repl.mean.mm=data.repl.mean
for (k in 1:ncol(data.repl.mean.mm)) {
  data.repl.mean.mm[,k]=(data.repl.mean.mm[,k]-min(data.repl.mean.mm[,k]))/(max(data.repl.mean.mm[,k])-min(data.repl.mean.mm[,k]))
  
}

data.repl.mean.mm.d=data.frame(data.repl.mean.mm,survive=as.factor(data$Survived))
train_control = trainControl(method = "cv", number = 10)
fit3.cv=train(survive~ .,
              data = data.repl.mean.mm.d,
              trControl = train_control,
              method = "glm",
              family=binomial(link='logit'))

acc3=fit3.cv$results[,2]


#min-max + mode
data.repl.mode.mm=data.repl.mode
for (k in 1:ncol(data.repl.mode.mm)) {
  data.repl.mode.mm[,k]=(data.repl.mode.mm[,k]-min(data.repl.mode.mm[,k]))/(max(data.repl.mode.mm[,k])-min(data.repl.mode.mm[,k]))
  
}

data.repl.mode.mm.d=data.frame(data.repl.mode.mm,survive=as.factor(data$Survived))
train_control = trainControl(method = "cv", number = 10)
fit4.cv=train(survive~ .,
              data = data.repl.mode.mm.d,
              trControl = train_control,
              method = "glm",
              family=binomial(link='logit'))

acc4=fit4.cv$results[,2]


##normal
# normal + mean
data.repl.mean.n=scale(data.repl.mean)
data.repl.mean.n.d=data.frame(data.repl.mean.n,survive=as.factor(data$Survived))

train_control = trainControl(method = "cv", number = 10)
fit5.cv=train(survive~ .,
              data = data.repl.mean.n.d,
              trControl = train_control,
              method = "glm",
              family=binomial(link='logit'))
acc5=fit5.cv$results[,2]

# normal + mode
data.repl.mode.n=scale(data.repl.mode)
data.repl.mode.n.d=data.frame(data.repl.mode.n,survive=as.factor(data$Survived))

train_control = trainControl(method = "cv", number = 10)
fit6.cv=train(survive~ .,
              data = data.repl.mode.n.d,
              trControl = train_control,
              method = "glm",
              family=binomial(link='logit'))
acc6=fit6.cv$results[,2]

result=data.frame(type=c('origin + mean',
                         'origin + mode',
                         'max-min + mean',
                         'max-min + mode',
                         'normal + mean',
                         'normal + mode'),accuracy=c(acc1,acc2,acc3,acc4,acc5,acc6))
result

Warning message:
"package 'ttutils' was built under R version 3.5.2"Warning message:
"package 'modes' was built under R version 3.5.2"Warning message:
"package 'caret' was built under R version 3.5.3"Loading required package: lattice
Warning message:
"package 'lattice' was built under R version 3.5.2"Loading required package: ggplot2


type,accuracy
origin + mean,0.6992634
origin + mode,0.6969087
max-min + mean,0.6992688
max-min + mode,0.6980956
normal + mean,0.7024889
normal + mode,0.6969788


## 整體來說，normal的表現比origin與max-min還要好，其中又以normal + mean  這個比現最好。